# PIP INSTALL AND IMPORT MODULE

In [ ]:
!pip install datasets evaluate transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup

from datasets import DatasetDict, Dataset
from sklearn.model_selection import train_test_split
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import ast

# We can see all columns in df.head() / and .tail()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', None)

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#this link
file_id = '1i5-lhcT6N2IOhTmKmfTaJmvSSQkgsFKA'

download = drive.CreateFile({'id': file_id})

# Download the file to a local disc
download.GetContentFile('tokenized_df.csv')

# PREPROCESSING


In [ ]:
df = pd.read_csv('tokenized_df.csv', engine='python')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546785 entries, 0 to 546784
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   content_text    546785 non-null  object
 1   input_ids       546785 non-null  object
 2   attention_mask  546785 non-null  object
 3   targets         546785 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 16.7+ MB


In [ ]:
df.head()

,content_text,input_ids,attention_mask,targets
0,Wall St. Pullback Reflects Tech Blowout (Reute...,"[101, 2813, 2358, 1012, 4139, 5963, 11138, 662...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
1,Wall St. Bears Claw Back Into the Black (Reute...,"[101, 2813, 2358, 1012, 6468, 15020, 2067, 204...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,"[101, 3514, 1998, 4610, 6112, 15768, 1005, 176...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
3,Iraq Halts Oil Exports from Main Southern Pipe...,"[101, 5712, 9190, 2015, 3514, 14338, 2013, 236...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
4,"Stocks End Up, But Near Year Lows (Reuters) Re...","[101, 15768, 2203, 2039, 1010, 2021, 2379, 209...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0


In [ ]:
df.rename(columns={'content_text': 'text', 'targets':'label'}, inplace=True)
df = df[['text', 'label', 'input_ids', 'attention_mask']]

In [ ]:
df['input_ids'] = df['input_ids'].apply(lambda x: ast.literal_eval(x))
df['attention_mask'] = df['attention_mask'].apply(lambda x: ast.literal_eval(x))

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=42)
train_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True,inplace=True)
val_df.reset_index(drop=True,inplace=True)

In [ ]:
train_df.head()

,text,label,input_ids,attention_mask
0,Japanese phone subscriptions top 100M (AP...,1,"[101, 2887, 3042, 15002, 2015, 2327, 2531, 221...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Edwards: I Won't Raise Retirement Age (AP) AP ...,4,"[101, 7380, 1024, 1045, 2180, 1005, 1056, 5333...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,RFID royalties suspended for testing Intermec ...,1,"[101, 21792, 3593, 25335, 6731, 2005, 5604, 69...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Federal Officials Scrutinizing Company Payment...,0,"[101, 2976, 4584, 8040, 22134, 5498, 6774, 219...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,Art Review | Richard Prince: Pilfering a Cultu...,4,"[101, 2396, 3319, 1064, 2957, 3159, 1024, 1425...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
val_df.head()

,text,label,input_ids,attention_mask
0,Attempt for closure stirs up more anger World ...,3,"[101, 3535, 2005, 8503, 16130, 2015, 2039, 206...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Is Finish Line Nearing a Flatline? The athleti...,0,"[101, 2003, 3926, 2240, 23454, 1037, 4257, 417...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,Four US Marines killed in Iraq Four US Marines...,4,"[101, 2176, 2149, 9622, 2730, 1999, 5712, 2176...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Weather Hinders Search for Ship's Crew as Oil ...,1,"[101, 4633, 17666, 2545, 3945, 2005, 2911, 100...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"Play it again, Bollywood (Weekly Roundup) The ...",2,"[101, 2377, 2009, 2153, 1010, 16046, 1006, 488...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
test_df.head()

,text,label,input_ids,attention_mask
0,UPDATE 3-SBC sees $5 billion in costs for fibe...,0,"[101, 10651, 1017, 1011, 24829, 2278, 5927, 10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Yahoo Local Officially Launches After just two...,1,"[101, 20643, 2334, 3985, 18989, 2044, 2074, 20...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"Police remove 209 cats, 3 dogs from home ...",1,"[101, 2610, 6366, 19348, 8870, 1010, 1017, 607...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"Baghdad Bombings Kill 35 Children BAGHDAD, Ira...",4,"[101, 13952, 20109, 3102, 3486, 2336, 13952, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"Up to 80 whales, dolphins die on Australia&#39...",4,"[101, 2039, 2000, 3770, 17967, 1010, 13600, 32...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
train_df.head()

,text,label,input_ids,attention_mask
0,Japanese phone subscriptions top 100M (AP...,1,"[101, 2887, 3042, 15002, 2015, 2327, 2531, 221...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Edwards: I Won't Raise Retirement Age (AP) AP ...,4,"[101, 7380, 1024, 1045, 2180, 1005, 1056, 5333...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,RFID royalties suspended for testing Intermec ...,1,"[101, 21792, 3593, 25335, 6731, 2005, 5604, 69...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Federal Officials Scrutinizing Company Payment...,0,"[101, 2976, 4584, 8040, 22134, 5498, 6774, 219...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,Art Review | Richard Prince: Pilfering a Cultu...,4,"[101, 2396, 3319, 1064, 2957, 3159, 1024, 1425...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
train = Dataset.from_pandas(train_df)
test = Dataset.from_pandas(test_df)
validation = Dataset.from_pandas(val_df)
transformed_dataset = DatasetDict({'train': train, 'validation': validation,'test': test})

In [ ]:
print(transformed_dataset)

In [ ]:
#map to map target numeric w actual values
id2label = {0:"Business", 1:"Sci/Tech", 2:"Entertainment", 3:"Sports", 4:"World"}
label2id = {"Business":0, "Sci/Tech":1, "Entertainment":2, "Sports":3, "World":4}

# COMPUTE METRIC FUNCTION

In [ ]:
import evaluate

accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average="weighted")
    recall = recall_metric.compute(predictions=predictions, references=labels, average="weighted")
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")


    return {"precision": precision,
        "recall": recall,
        "f1": f1,
        "accuracy": accuracy}

# TRAINING

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "nghuyong/ernie-2.0-base-en", num_labels=5, id2label=id2label, label2id=label2id
)

Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('nghuyong/ernie-2.0-base-en')
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
transformed_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 382749
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 82018
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 82018
    })
})

In [ ]:
training_args = TrainingArguments(
    output_dir="ernie-base-2.0-en-agnews",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=transformed_dataset["train"],
    eval_dataset=transformed_dataset["validation"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.391600,0.381009,{'precision': 0.8517771242212259},{'recall': 0.8496671462361921},{'f1': 0.8452058339265478},{'accuracy': 0.8496671462361921}
2,0.314700,0.377601,{'precision': 0.8522103447774182},{'recall': 0.8533980345777756},{'f1': 0.8506811267748378},{'accuracy': 0.8533980345777756}
3,0.282000,0.391321,{'precision': 0.8506427578646623},{'recall': 0.8524226389329171},{'f1': 0.8498698821903607},{'accuracy': 0.8524226389329171}


Trainer is attempting to log a value of "{'precision': 0.8517771242212259}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.8496671462361921}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.8452058339265478}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.8496671462361921}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8522103447774182}" of type <class

TrainOutput(global_step=71766, training_loss=0.35700462953012413, metrics={'train_runtime': 7599.0807, 'train_samples_per_second': 151.103, 'train_steps_per_second': 9.444, 'total_flos': 5.90087144003634e+16, 'train_loss': 0.35700462953012413, 'epoch': 3.0})

# OPTUNA WITH LOADED CHECKPOINTS

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("tamhuynh27/ernie-base-2.0-en-agnews", num_labels=5, id2label=id2label, label2id=label2id)
tokenizer = AutoTokenizer.from_pretrained("tamhuynh27/ernie-base-2.0-en-agnews")


In [ ]:
import evaluate
import transformers
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
import optuna
from transformers import Trainer, TrainingArguments


def objective(trial):
    # Hyperparameters to be optimized
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 3e-5, log=True)
    num_train_epochs = trial.suggest_int("num_train_epochs", 1, 3)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [8, 16])
    weight_decay = trial.suggest_float("weight_decay", 0.05, 0.1)
    temperature = trial.suggest_int("temperature", 2, 10)

    # TrainingArguments & Trainer
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        learning_rate=learning_rate,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        weight_decay=weight_decay,

    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=transformed_dataset["train"],
        eval_dataset=transformed_dataset["validation"],
        compute_metrics=compute_metrics,
        tokenizer=tokenizer
    )

    # Training
    trainer.train()

    # Evaluation
    metrics = trainer.evaluate()

    return metrics['eval_accuracy']


In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)  # Number of hyperparameter optimization trials

# Results
print('Number of finished trials: ', len(study.trials))
print('Best trial:')
trial = study.best_trial

print('Value: ', trial.value)
print('Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')


[I 2023-10-26 20:00:32,492] A new study created in memory with name: no-name-6ccf9c13-8801-4f51-b115-a15697b7b501
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.376200,0.479778,0.847570
2,0.342800,0.478876,0.846692
3,0.260700,0.562235,0.838499


[I 2023-10-26 22:18:17,014] Trial 0 finished with value: 0.8466921895193738 and parameters: {'learning_rate': 1.9491892388121254e-05, 'num_train_epochs': 3, 'per_device_train_batch_size': 8, 'weight_decay': 0.07356131937034624, 'temperature': 5}. Best is trial 0 with value: 0.8466921895193738.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.242300,0.629919,0.832500


[I 2023-10-26 23:05:51,330] Trial 1 finished with value: 0.8325001828866834 and parameters: {'learning_rate': 1.319734927080758e-05, 'num_train_epochs': 1, 'per_device_train_batch_size': 8, 'weight_decay': 0.09362462472991712, 'temperature': 7}. Best is trial 0 with value: 0.8466921895193738.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.325300,0.592079,0.836329
2,0.279900,0.692461,0.821149


[I 2023-10-27 00:38:18,477] Trial 2 finished with value: 0.8363286107927528 and parameters: {'learning_rate': 2.4477222958661536e-05, 'num_train_epochs': 2, 'per_device_train_batch_size': 8, 'weight_decay': 0.07148803145477037, 'temperature': 5}. Best is trial 0 with value: 0.8466921895193738.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.309700,0.697502,0.831866
2,0.288600,0.704002,0.818162


[I 2023-10-27 02:10:50,476] Trial 3 finished with value: 0.8318661757175254 and parameters: {'learning_rate': 2.6498064468633915e-05, 'num_train_epochs': 2, 'per_device_train_batch_size': 8, 'weight_decay': 0.08099433779986762, 'temperature': 7}. Best is trial 0 with value: 0.8466921895193738.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.276600,0.887571,0.823429
2,0.288600,0.725324,0.812395
